In [4]:
%load_ext autoreload
%autoreload 2

In [1]:
#common libs
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader
from torchinfo import summary
import math
from easydict import EasyDict as edict


In [3]:
import torch
import torch.nn as nn

lstm = nn.LSTM(input_size=3, hidden_size=3, num_layers=1, batch_first=True)

x = torch.randn(2, 5, 3)  # random input: batch of 2 sequences, each length 5, features 3

output, (h_n, c_n) = lstm(x)
print("Output shape:", output.shape)  # Output shape: (2, 5, 4)

Output shape: torch.Size([2, 5, 3])


In [2]:
#mtr modules
from mtr.datasets import build_dataloader
from mtr.config import cfg, cfg_from_yaml_file
from mtr.utils import common_utils

In [3]:
cfg_from_yaml_file("/code/jjiang23/csc587/KimchiVision/cfg/kimchiConfig.yaml", cfg)
logger = common_utils.create_logger("/files/waymo/damon_log.txt", rank=0)
args = edict({
    "batch_size": 8,
    "workers": 32,
    "merge_all_iters_to_one_epoch": False,
    "epochs": 5,
    "add_worker_init_fn": False,
    
})

In [5]:
#prepare data
train_set, train_loader, train_sampler = build_dataloader(
    dataset_cfg=cfg.DATA_CONFIG,
    batch_size=args.batch_size,
    dist=False, workers=args.workers,
    logger=logger,
    training=True,
    merge_all_iters_to_one_epoch=args.merge_all_iters_to_one_epoch,
    total_epochs=args.epochs,
    add_worker_init_fn=args.add_worker_init_fn,
)

test_set, test_loader, sampler = build_dataloader(
        dataset_cfg=cfg.DATA_CONFIG,
        batch_size=args.batch_size,
        dist=False, workers=args.workers, logger=logger, training=False
)

2025-06-08 22:02:13,752   INFO  Start to load infos from /files/waymo/code/MTR/data/waymo/processed_scenarios_training_infos.pkl


2025-06-08 22:02:19,582   INFO  Total scenes before filters: 243401
2025-06-08 22:02:26,791   INFO  Total scenes after filter_info_by_object_type: 243401
2025-06-08 22:02:26,816   INFO  Total scenes after filters: 243401
2025-06-08 22:02:26,997   INFO  Start to load infos from /files/waymo/code/MTR/data/waymo/processed_scenarios_val_infos.pkl
2025-06-08 22:02:28,708   INFO  Total scenes before filters: 22089
2025-06-08 22:02:29,454   INFO  Total scenes after filter_info_by_object_type: 22089
2025-06-08 22:02:29,456   INFO  Total scenes after filters: 22089


In [6]:
from lstm.train_util import train_model
from lstm.lstm import MotionLSTM

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MotionLSTM().to(device)

In [7]:
batch = next(iter(train_loader))

In [ ]:
batch["input_dict"].keys()

In [ ]:
input = batch["input_dict"]
obj_trajs = input["obj_trajs"]
obj_pos = input["obj_trajs_pos"]
obj_last_pos = input["obj_trajs_last_pos"]
obj_type = input["obj_types"] # car, bicycycle, pedestrian
obj_trajs_mask = input['obj_trajs_mask']
obj_of_interest = input['track_index_to_predict']

In [ ]:
num_center_objects, num_objects, num_timestamps, num_attrs = obj_trajs.shape

In [ ]:
static_map_polylines=input["static_map_polylines"].to('cuda')  # (batch_size, num_polylines, num_points_each_polyline, 7)
static_map_polylines_mask=input["static_map_polylines_mask"].to('cuda') # (batch_size, num_polylines, num_points_each_polyline)

In [ ]:
model._print_batch(batch)

In [ ]:
batch["input_dict"]["center_gt_final_valid_idx"]

In [ ]:
torch.nonzero(obj_trajs_mask[0, 0, :])[-1].squeeze()

In [ ]:
from lstm.loss import MotionLoss

criterion = MotionLoss()

In [ ]:
batch["input_dict"]["obj_trajs_mask"][0, 0, :][-1].squeeze()

In [8]:
pred_scores, pred_trajs = model(batch)

0 torch.Size([30, 96, 11]) tensor(11, device='cuda:0')
1 torch.Size([30, 96, 11]) tensor(11, device='cuda:0')
2 torch.Size([30, 96, 11]) tensor(11, device='cuda:0')
3 torch.Size([30, 96, 11]) tensor(11, device='cuda:0')
4 torch.Size([30, 96, 11]) tensor(11, device='cuda:0')
5 torch.Size([30, 96, 11]) tensor(11, device='cuda:0')
6 torch.Size([30, 96, 11]) tensor(11, device='cuda:0')
7 torch.Size([30, 96, 11]) tensor(11, device='cuda:0')
8 torch.Size([30, 96, 11]) tensor(11, device='cuda:0')
9 torch.Size([30, 96, 11]) tensor(11, device='cuda:0')
10 torch.Size([30, 96, 11]) tensor(11, device='cuda:0')
11 torch.Size([30, 96, 11]) tensor(11, device='cuda:0')
12 torch.Size([30, 96, 11]) tensor(11, device='cuda:0')
13 torch.Size([30, 96, 11]) tensor(11, device='cuda:0')
14 torch.Size([30, 96, 11]) tensor(11, device='cuda:0')
15 torch.Size([30, 96, 11]) tensor(11, device='cuda:0')
16 torch.Size([30, 96, 11]) tensor(11, device='cuda:0')
17 torch.Size([30, 96, 11]) tensor(11, device='cuda:0')
18

IndexError: index -1 is out of bounds for dimension 0 with size 0

In [9]:
pred_scores.shape

torch.Size([1, 6])

In [ ]:
loss

In [ ]:
print(loss)

In [ ]:
input['center_gt_trajs_mask'].shape

In [ ]:
#define loss function
class MotionLoss(nn.Module):
    """
    Loss function for trajectory prediction
    """
    
    def __init__(self, 
                 regression_loss_weight=1.0,
                 classification_loss_weight=1.0,
                 future_loss_weight=1.0):
        super(MotionLoss, self).__init__()
        self.reg_weight = regression_loss_weight
        self.cls_weight = classification_loss_weight
        self.future_weight = future_loss_weight
    
    def forward(self, pred_scores, pred_trajs, batch_dict):
        """
        Compute loss
        
        Args:
            pred_scores: (batch_size, num_modes)
            pred_trajs: (batch_size, num_modes, future_steps, 4)
            batch_dict: Contains ground truth data
        
        Returns:
            loss_dict: Dictionary containing different loss components
        """
        center_gt_trajs = batch_dict['input_dict']['center_gt_trajs'].to('cuda')  # (batch_size, future_steps, 4)
        center_gt_trajs_mask = batch_dict['input_dict']['center_gt_trajs_mask'].to('cuda')  # (batch_size, future_steps)
        
        batch_size, num_modes, future_steps, _ = pred_trajs.shape
        
        # Compute trajectory regression loss for each mode
        gt_trajs_expanded = center_gt_trajs.unsqueeze(1).expand(-1, num_modes, -1, -1)
        gt_mask_expanded = center_gt_trajs_mask.unsqueeze(1).expand(-1, num_modes, -1)
        
        # L2 loss for position (x, y)
        pos_loss = F.mse_loss(
            pred_trajs[:, :, :, :2] * gt_mask_expanded.unsqueeze(-1),
            gt_trajs_expanded[:, :, :, :2] * gt_mask_expanded.unsqueeze(-1),
            reduction='none'
        ).sum(dim=-1)  # (batch_size, num_modes, future_steps)
        
        # L2 loss for velocity (vx, vy)
        vel_loss = F.mse_loss(
            pred_trajs[:, :, :, 2:4] * gt_mask_expanded.unsqueeze(-1),
            gt_trajs_expanded[:, :, :, 2:4] * gt_mask_expanded.unsqueeze(-1),
            reduction='none'
        ).sum(dim=-1)  # (batch_size, num_modes, future_steps)
        
        # Weighted loss over time (give more weight to near future)
        time_weights = torch.exp(-0.1 * torch.arange(future_steps, device=pred_trajs.device))
        time_weights = time_weights.view(1, 1, -1)
        
        pos_loss = (pos_loss * time_weights * gt_mask_expanded).sum(dim=-1)  # (batch_size, num_modes)
        vel_loss = (vel_loss * time_weights * gt_mask_expanded).sum(dim=-1)  # (batch_size, num_modes)
        
        # Find best mode for each sample
        total_traj_loss = pos_loss + vel_loss  # (batch_size, num_modes)
        best_mode_indices = torch.argmin(total_traj_loss, dim=1)  # (batch_size,)
        
        # Regression loss (best mode)
        best_pos_loss = pos_loss[torch.arange(batch_size), best_mode_indices].mean()
        best_vel_loss = vel_loss[torch.arange(batch_size), best_mode_indices].mean()
        regression_loss = best_pos_loss + best_vel_loss
        
        # Classification loss (encourage higher confidence for best mode)
        target_scores = torch.zeros_like(pred_scores)
        target_scores[torch.arange(batch_size), best_mode_indices] = 1.0
        classification_loss = F.cross_entropy(pred_scores, target_scores)
        
        # Total loss
        total_loss = (self.reg_weight * regression_loss + 
                     self.cls_weight * classification_loss)
        
        loss_dict = {
            'total_loss': total_loss,
            'regression_loss': regression_loss,
            'classification_loss': classification_loss,
            'pos_loss': best_pos_loss,
            'vel_loss': best_vel_loss
        }
        
        return loss_dict


In [ ]:
from lstm.loss import MotionLoss

#train loop
def train_model(model, train_dataloader, val_dataloader, num_epochs=100, lr=1e-3):
    """
    Training loop for the LSTM model
    """
    assert torch.cuda.is_available(), "CUDA is not available. Please check your PyTorch installation."
    device = torch.device('cuda')
    model.to(device)
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
    criterion = MotionLoss()
    
    best_val_loss = float('inf')
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_losses = []
        
        for batch_idx, batch_dict in enumerate(train_dataloader):
            # Move data to device
            for key, value in batch_dict.items():
                if isinstance(value, torch.Tensor):
                    batch_dict[key] = value.to(device)
            
            optimizer.zero_grad()
            
            # Forward pass
            pred_scores, pred_trajs = model(batch_dict)
            
            # Compute loss
            loss = criterion(pred_scores, pred_trajs, batch_dict)
            
            # Backward pass
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            train_losses.append(loss.item())
            
            if batch_idx % 100 == 0:
                print(f'Epoch {epoch}, Batch {batch_idx}, Loss: {loss.item():.4f}')
        
        # Validation phase
        model.eval()
        val_losses = []
        
        with torch.no_grad():
            for batch_dict in val_dataloader:
                # Move data to device
                for key, value in batch_dict.items():
                    if isinstance(value, torch.Tensor):
                        batch_dict[key] = value.to(device)
                
                pred_scores, pred_trajs = model(batch_dict)
                loss_dict = criterion(pred_scores, pred_trajs, batch_dict)
                val_losses.append(loss_dict['total_loss'].item())
        
        scheduler.step()
        
        avg_train_loss = np.mean(train_losses)
        avg_val_loss = np.mean(val_losses)
        
        print(f'Epoch {epoch}: Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')
        
        # Save best model
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), '/code/jjiang23/csc587/KimchiVision/best_motion_lstm.pth')
    
    return model

In [ ]:
from lstm.train_util import train_model
from lstm.lstm import MotionLSTM

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MotionLSTM().to(device)

# Train the model
trained_model = train_model(model, train_loader, test_loader)